In [1]:
import json
import time
import math
from cerebellum.data_prep.seg_prep import *

resolution = (30, 48, 48)
block_size = 60
affinity_offset = 14
wz_thresh = 0.5
n_blocks = 15
block_ids = range(0,n_blocks)
block_shape = (block_size*n_blocks, 540, 489)
superblock_steps = [2, 4, 8, 12, 16] # save after assembling these many blocks

/home/srujanm/anaconda2/envs/em/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# assemble linearly generated blockchain linked to 0 block
save_folder = './superblocks/'
create_folder(save_folder)
save_fname = 'waterz%.2f-48nm-crop2gt-linear-superblock'%(wz_thresh)
total_runtime = 0
superblock = np.zeros(block_shape, dtype=np.uint32) 
for block_id in block_ids:
    block_time = time.time()
    zz = block_id*block_size + affinity_offset
    block_name = "waterz%.2f-48nm-crop2gt-%04d"%(wz_thresh, zz)
    print "Appending the %dth block: "%(block_id) + block_name
    block = SegPrep(block_name, resolution)
    if block_id==0: block.read_internal(stage="filtered")
    else: block.read_internal(stage="locked-to-0")
    print "Finished reading block"
    superblock[block_id*block_size:(block_id+1)*block_size,
               :,:] = block.data
    block_runtime = time.time()-block_time
    print "Finished operations with block %d in %f seconds"%(block_id, block_runtime)
    total_runtime += block_runtime
    if block_id+1 in superblock_steps:
        print "Saving superblock with %d blocks"%(block_id+1)
        writeh5(save_folder+save_fname+ '-%dblocks.h5'%(block_id+1), 'main', 
                superblock[:(block_id+1)*block_size,:,:], compression='gzip')
print "Completed assembling superblocks. Total runtime: %f"%(total_runtime)

Appending the 0th block: waterz0.50-48nm-crop2gt-0014
Finished reading block
Finished operations with block 0 in 0.819967 seconds
Appending the 1th block: waterz0.50-48nm-crop2gt-0074
Finished reading block
Finished operations with block 1 in 0.789091 seconds
Saving superblock with 2 blocks
Appending the 2th block: waterz0.50-48nm-crop2gt-0134
Finished reading block
Finished operations with block 2 in 0.782283 seconds
Appending the 3th block: waterz0.50-48nm-crop2gt-0194
Finished reading block
Finished operations with block 3 in 0.792756 seconds
Saving superblock with 4 blocks
Appending the 4th block: waterz0.50-48nm-crop2gt-0254
Finished reading block
Finished operations with block 4 in 0.804029 seconds
Appending the 5th block: waterz0.50-48nm-crop2gt-0314
Finished reading block
Finished operations with block 5 in 0.964636 seconds
Appending the 6th block: waterz0.50-48nm-crop2gt-0374
Finished reading block
Finished operations with block 6 in 0.799826 seconds
Appending the 7th block: w

In [4]:
# generate downsampled full superblock for visualization
resolution_vis = (96,96,30)
superblock_vis = superblock
print superblock_vis.shape
print "Generating downsampled full superblock for ng visualization"
# zero every 3rd z-slice, so effective z-resolution is 90nm
for z in range(0,superblock_vis.shape[0],3):
    superblock_vis[z,:,:] = 0
writeh5(save_folder+save_fname+'_vis.h5', 'main', superblock_vis, compression="gzip")

(900, 540, 489)
Generating downsampled full superblock for ng visualization
